# 02_01_DAOStarfinder

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, astropy, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name==version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

In [5]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, imexam, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        print(f"**** module {pkg} is not installed... now start install")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"****** module {pkg} is installed")
    else: 
        print(f"**** module {pkg} is installed")

**** module numpy is installed
**** module pandas is installed
**** module matplotlib is installed
**** module scipy is installed
**** module astropy is installed
**** module photutils is installed
**** module ccdproc is installed
**** module imexam is installed
**** module version_information is installed


### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [6]:
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2023-02-01 20:04:00 (KST = GMT+0900) 
0 Python     3.8.16 64bit [GCC 11.2.0]
1 IPython    8.8.0
2 OS         Linux 5.15.0 58 generic x86_64 with glibc2.17
3 numpy      1.22.3
4 pandas     1.5.2
5 matplotlib 3.6.3
6 scipy      1.8.1
7 astropy    5.2.1
8 photutils  1.6.0
9 ccdproc    2.4.0
10 imexam     0.9.1
11 version_information 1.0.4


### import modules

In [7]:
import os
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rcParams
from mpl_toolkits.axes_grid1 import make_axes_locatable

import ysfitsutilpy as yfu
import ysphotutilpy as ypu
import ysvisutilpy as yvu

import _Python_utilities
import _astro_utilities

from astropy.nddata import Cutout2D
from astropy.io import fits
from astropy.wcs import WCS

from photutils import DAOStarFinder
from photutils import IRAFStarFinder
from photutils.aperture import CircularAperture as CAp
from photutils.aperture import CircularAnnulus as CAn
from photutils import detect_threshold
from photutils.centroids import centroid_com
#from photutils import aperture_photometry as apphot

import warnings

from ccdproc import CCDData, ccd_process

from astropy.time import Time
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord

plt.rcParams.update({'figure.max_open_warning': 0})

/home/guitar79/Downloads/ysphotutilpy/ysphotutilpy/seputil.py:112: UserWarning: Package sep is not installed. Some functions will not work.
  warn("Package sep is not installed. Some functions will not work.")
/tmp/ipykernel_50607/2468915714.py:22: DeprecationWarning: `photutils.DAOStarFinder` is a deprecated alias for `photutils.detection.DAOStarFinder` and will be removed in the future. Instead, please use `from photutils.detection import DAOStarFinder` to silence this warning.
  from photutils import DAOStarFinder
/tmp/ipykernel_50607/2468915714.py:23: DeprecationWarning: `photutils.IRAFStarFinder` is a deprecated alias for `photutils.detection.IRAFStarFinder` and will be removed in the future. Instead, please use `from photutils.detection import IRAFStarFinder` to silence this warning.
  from photutils import IRAFStarFinder
/tmp/ipykernel_50607/2468915714.py:26: DeprecationWarning: `photutils.detect_threshold` is a deprecated alias for `photutils.segmentation.detect_threshold` and 

In [8]:
#%%
BASEDIR = _astro_utilities.base_dir

BASEDIRs = sorted(_Python_utilities.getFullnameListOfsubDir(BASEDIR))
print ("BASEDIRs: {}".format(BASEDIRs))
print ("len(BASEDIRs): {}".format(len(BASEDIRs)))


BASEDIR = Path(BASEDIRs[0])
print ("Starting...\n{}".format(BASEDIR))

BASEDIR = Path(BASEDIR)

BASEDIR = Path(BASEDIR)
SOLVEDDIR = BASEDIR / _astro_utilities.solved_dir2
DAORESULTDIR = BASEDIR / _astro_utilities.DAOfinder_result_dir

if not DAORESULTDIR.exists():
    os.makedirs("{}".format(str(DAORESULTDIR)))
    print("{} is created...".format(str(DAORESULTDIR)))

BASEDIRs: ['/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-24_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-25_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-27_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-11-02_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-11-04_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-11-17_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/LANDOLT-114670_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/LANDOLT-114670_Light_-_2022-10-24_-_RiLA600_

In [9]:
#summary = yfu.make_summary(BASEDIR/"*.fit*")
summary = yfu.make_summary(SOLVEDDIR/"*.fit*")

if summary.empty:
        print("The dataframe(summary) is empty")
        pass
else:
    print("len(summary):", len(summary))
    print("summary:", summary)

All 123 keywords (guessed from /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/solved2/KLEOPATRA_Light_b_2022-10-23-10-58-26_180sec_RiLA600_STX-16803_-30C_2bin.fit) will be loaded.


/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:298: UserWarning: Key CDELT1 not found for /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/solved2/KLEOPATRA_Light_b_2022-10-23-11-07-09_180sec_RiLA600_STX-16803_-30C_2bin.fit, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:298: UserWarning: Key CDELT2 not found for /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/solved2/KLEOPATRA_Light_b_2022-10-23-11-07-09_180sec_RiLA600_STX-16803_-30C_2bin.fit, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:298: UserWarning: Key CROTA1 not found for /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/solved2/KLEOPATRA_Light_b_2022-10-23-11-07-09_180sec_RiLA

len(summary): 64
summary:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
1   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
2   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
3   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
4   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
..                                                ...       ...     ...   
59  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   
60  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   
61  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   
62  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   
63  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   

    BITPIX  NAXIS  NAXIS1  NAXIS2             FITS-TLM IMAGETYP  EXPOSURE

/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:298: UserWarning: Key PLTSOLVD not found for /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/solved2/KLEOPATRA_Light_v_2022-10-23-13-40-19_120sec_RiLA600_STX-16803_-30C_2bin.fit, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:298: UserWarning: Key _CSAXES not found for /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/solved2/KLEOPATRA_Light_v_2022-10-23-13-40-19_120sec_RiLA600_STX-16803_-30C_2bin.fit, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:298: UserWarning: Key _RPIX1 not found for /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/solved2/KLEOPATRA_Light_v_2022-10-23-13-40-19_120sec_R

### Light

In [10]:
df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
df_light = df_light.reset_index(drop=True)
print("df_light:\n{}".format(df_light))


df_light:
                                                 file  filesize  SIMPLE  \
0   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
1   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
2   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
3   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
4   /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16845120    True   
..                                                ...       ...     ...   
59  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   
60  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   
61  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   
62  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   
63  /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_...  16842240    True   

    BITPIX  NAXIS  NAXIS1  NAXIS2             FITS-TLM IMAGETYP  EXPOSURE  \
0      -32  

In [11]:
for _, row  in df_light.iterrows():
    fpath = Path(row["file"])
    print("type(fpath)", type(fpath))
    print("fpath", fpath)

    hdul = fits.open(fpath)

    hdr = hdul[0].header
    img = hdul[0].data
    print("img: {}".format(img))
    print("img.shape: {}".format(img.shape))

    # Set WCS and print for your information
    w = WCS(hdr)
    print("WCS: {}".format(w))

    thresh = detect_threshold(data=img, nsigma=3)
    thresh = thresh[0][0]
    print('detect_threshold', thresh)

    #%%
    try:
        
        FWHM   = 6
        
        IRAFfind = IRAFStarFinder(
                                fwhm = FWHM, 
                                threshold = thresh,
                                sigma_radius = 1.5, minsep_fwhm = 2.5,  # default values: sigma_radius=1.5, minsep_fwhm=2.5,
                                sharplo = 0.5, sharphi = 2.0,   # default values: sharplo=0.5, sharphi=2.0,
                                roundlo = 0.0, roundhi = 0.2,   # default values: roundlo=0.0, roundhi=0.2,
                                sky = None,                     # default values: sky=None
                                exclude_border = True  # default values: exclude_border=False
                                )
                            
        # The IRAFStarFinder object ("IRAFfind") gets at least one input: the image.
        # Then it returns the astropy table which contains the aperture photometry results:
        IRAFfound = IRAFfind(img)
        print('{} star(s) founded by IRAFStarFinder...'.format(len(IRAFfound)))

        #%%
        if len(IRAFfound)==0 :
            print ('No star was founded by IRAFStarFinder...\n'*3)
        else : 

            # Use the object "found" for aperture photometry:
            N_stars = len(IRAFfound)
            print('{} star(s) founded by IRAFStarFinder...'.format(N_stars))
            IRAFfound.pprint(max_width=1800)

            # save XY coordinates:
            IRAFfound.write("{}/{}_IRAFStarfinder_fwhm{}.csv".\
                            format(IRAFRESULTDIR, fpath.stem, FWHM), 
                            overwrite = True,
                            format='ascii.fast_csv')

            print('type(IRAFfound): {}'.format(type(IRAFfound)))
            print('IRAFfound: {}'.format(IRAFfound))

            IRAFcoord = np.array([IRAFfound['xcentroid'], IRAFfound['ycentroid']]).T
            print('type(IRAFcoord): {}'.format(type(IRAFcoord)))
            print('IRAFcoord: {}'.format(IRAFcoord))

            #%%
            # Save apertures as circular, 4 pixel radius, at each (X, Y)
            IRAFapert = CAp((IRAFcoord), r=4.)  
            print('type(IRAFapert): {}'.format(type(IRAFapert)))
            print('IRAFapert: {}'.format(IRAFapert))
            print('dir(IRAFapert): {}'.format(dir(IRAFapert)))

            IRAFannul = CAn(positions = (IRAFcoord), r_in = 4*FWHM, r_out = 6*FWHM) 
            print('type(IRAFannul): {}'.format(type(IRAFannul)))
            print('IRAFannul: {}'.format(IRAFannul))

            #%%
            plt.figure(figsize=(20,20))
            ax = plt.gca()

            ###########################################################
            # input some text for explaination. 
            plt.title("Result of IRAFStarfinder", fontsize = 28, 
                ha='center')

            plt.annotate('filename: {}'.format(fpath.stem), fontsize=10,
                xy=(1, 0), xytext=(-500, -40), va='top', ha='left',
                xycoords='axes fraction', textcoords='offset points')
                        
            plt.annotate('FWHM: {}'.format(FWHM), fontsize=10,
                xy=(1, 0), xytext=(-1100, -30), va='top', ha='left',
                xycoords='axes fraction', textcoords='offset points')
                
            plt.annotate('Sky threshold: {:02f}'.format(thresh), fontsize=10,
                xy=(1, 0), xytext=(-1100, -40), va='top', ha='left',
                xycoords='axes fraction', textcoords='offset points')

            plt.annotate('Number of star(s): {}'.format(len(IRAFfound)), fontsize=10,
                xy=(1, 0), xytext=(-1100, -50), va='top', ha='left',
                xycoords='axes fraction', textcoords='offset points')

            im = plt.imshow(img, 
                            vmin = thresh, 
                            vmax = thresh * 3,
                            #zscale=True,
                            origin='lower'
                            )

            IRAFannul.plot(color='red', lw=2., alpha=0.4)
            
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="3%", pad=0.05)
            plt.colorbar(im, cax=cax)

            plt.savefig(
                        "{}/{}_IRAFStarfinder_fwhm{}.png".\
                            format(IRAFRESULTDIR, fpath.stem, FWHM)
                        )
            print("{}/{}_IRAFStarfinder_fwhm{}.png is created...".\
                            format(IRAFRESULTDIR, fpath.stem, FWHM))
            #plt.show()
            plt.close() 

    except Exception as err:
        print('{0} with {1} '.format(err, fpath.name))

type(fpath) <class 'pathlib.PosixPath'>
fpath /mnt/Rdata/CCD_obs/RnE_2022/RiLA600_STX-16803_2bin/KLEOPATRA_Light_-_2022-10-23_-_RiLA600_STX-16803_-_2bin/solved2/KLEOPATRA_Light_b_2022-10-23-10-58-26_180sec_RiLA600_STX-16803_-30C_2bin.fit
img: [[7274.3535 7032.1226 7136.9775 ... 7007.1187 7080.712  6745.861 ]
 [7049.901  7102.915  7093.267  ... 7080.712  7020.0503 7237.7964]
 [7091.767  7123.0996 6956.123  ... 7142.93   7107.3887 7055.732 ]
 ...
 [7090.0234 7095.129  7104.7183 ... 7139.4585 7036.087  7062.32  ]
 [7071.743  7067.665  7131.3735 ... 7170.674  7054.696  6881.689 ]
 [7077.713  7167.15   7049.8145 ... 7115.597  7119.8423 7090.446 ]]
img.shape: (2048, 2048)
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated her

/home/guitar79/anaconda3/envs/astro_Python_ubuntu_env/lib/python3.8/site-packages/astropy/wcs/wcs.py:3064: RuntimeWarning: cdelt will be ignored since cd is present
  description.append(s.format(*self.wcs.cdelt))


detect_threshold 7299.290100097656
5 star(s) founded by IRAFStarFinder...
5 star(s) founded by IRAFStarFinder...
 id     xcentroid          ycentroid             fwhm            sharpness           roundness              pa         npix    sky         peak             flux               mag        
--- ------------------ ------------------ ------------------ ------------------ ------------------- ------------------ ---- --------- -------------- ----------------- -------------------
  1  1407.476506072681 333.62873755159467  3.366749622493468 0.5611249370822446 0.09768054760663186 61.413366716823006   42  9979.699     39478.1875 630334.0751953125 -14.498926963154695
  2 1725.5648253532754  451.9719771146101 3.3507273997283757 0.5584545666213959 0.14058239935813463  68.70569445831804   42  9539.845    33430.84375 523111.9580078125 -14.296486619593773
  3    724.46679808967    988.97459631841 3.3329649758927684 0.5554941626487947 0.18398335344447958  87.97543510155454   40  8449.719     1

/home/guitar79/anaconda3/envs/astro_Python_ubuntu_env/lib/python3.8/site-packages/astropy/wcs/wcs.py:3064: RuntimeWarning: cdelt will be ignored since cd is present
  description.append(s.format(*self.wcs.cdelt))


detect_threshold 7233.091178894043
7 star(s) founded by IRAFStarFinder...
7 star(s) founded by IRAFStarFinder...
 id     xcentroid          ycentroid             fwhm            sharpness           roundness               pa         npix    sky          peak             flux                mag        
--- ------------------ ------------------ ------------------ ------------------ -------------------- ------------------ ---- --------- --------------- ------------------ -------------------
  1 1403.1695623874891 345.01017041187583  3.458031225504626 0.5763385375841044  0.11212012764876925  34.14833150277566   44  9971.796   41112.8828125  647249.6865234375 -14.527679621922648
  2 1721.2091378869386 463.41331923326635 3.3798822629682514 0.5633137104947086  0.12150089778467353 36.621117661544005   44  9759.964  34782.82421875  520160.4287109375 -14.290343275233239
  3 216.80914412524658  828.6366223256115 3.3642863705559556 0.5607143950926593  0.15621280839370275 114.16573718633609   44 77

...

/home/guitar79/anaconda3/envs/astro_Python_ubuntu_env/lib/python3.8/site-packages/astropy/wcs/wcs.py:3064: RuntimeWarning: cdelt will be ignored since cd is present
  description.append(s.format(*self.wcs.cdelt))


detect_threshold 6906.142120361328
6 star(s) founded by IRAFStarFinder...
6 star(s) founded by IRAFStarFinder...
 id     xcentroid          ycentroid             fwhm            sharpness           roundness               pa         npix   sky          peak              flux                mag        
--- ------------------ ------------------ ------------------ ------------------ -------------------- ------------------ ---- -------- ---------------- ------------------ -------------------
  1  1391.253193198623  384.9807444392074    3.4691989879583 0.5781998313263833  0.16452005516068324  47.76200699286525   43 9650.746   36471.21484375  639068.4853515625 -14.513868503833143
  2 1709.2248117967442  503.5061452884865 3.3318486615115432 0.5553081102519238  0.16805970602203277  45.55719158978379   41 9836.909    31092.3984375   487201.560546875 -14.219271676770438
  3  204.7675723610348   867.876292136379 3.4123989313319436 0.5687331552219906  0.14159058840921696 105.81427491132159   45 74

/home/guitar79/anaconda3/envs/astro_Python_ubuntu_env/lib/python3.8/site-packages/astropy/wcs/wcs.py:3064: RuntimeWarning: cdelt will be ignored since cd is present
  description.append(s.format(*self.wcs.cdelt))


detect_threshold 6864.111930847168
9 star(s) founded by IRAFStarFinder...
9 star(s) founded by IRAFStarFinder...
 id     xcentroid          ycentroid             fwhm            sharpness           roundness              pa         npix    sky          peak              flux               mag        
--- ------------------ ------------------ ------------------ ------------------ ------------------- ------------------ ---- --------- ---------------- ----------------- -------------------
  1 1389.3346529423093 394.48680397984964  3.341715858652866  0.556952643108811 0.13499793960054932 48.321742845886455   42 10407.709      35085.65625  570522.689453125  -14.39068230202732
  2  1707.321767177994   513.188374870074  3.422716298376384  0.570452716396064  0.1653365880157731  55.05652853772809   44  9540.875     29524.234375  489835.150390625 -14.225124866798936
  3  202.8668960932214  877.1928137783725  3.425445033161264 0.5709075055268773 0.14377783412047884 106.34415577418842   45  7331.0

detect_threshold 6811.504364013672
